# Staatsangehörigkeit


In [ ]:
import plotly.io as pio
import pandas as pd
import plotly.express as px

infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text="",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template

## Daten einlesen

In [ ]:
file_path = "data.xlsx"
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")
sheets.pop(next(iter(sheets)))  
staatsangehoerigkeit = {"Jahr": [], "Deutsche Staatsangehörigkeit (%)": [], "Andere Staatsangehörigkeit (%)": []}

for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()
    
    deutsch_row = df[df["Variable"].str.contains("Staatsangehörigkeit", na=False) & 
                     df["Category"].str.contains("Deutsche Staatsangehörigkeit", na=False)]
    
    andere_row = df[df["Variable"].str.contains("Staatsangehörigkeit", na=False) & 
                    df["Category"].str.contains("Andere Staatsangehörigkeit", na=False)]
    
    if not deutsch_row.empty and not andere_row.empty:
        staatsangehoerigkeit["Jahr"].append(jahr)
        staatsangehoerigkeit["Deutsche Staatsangehörigkeit (%)"].append(deutsch_row.iloc[:, 2:].mean(axis=1).values[0])
        staatsangehoerigkeit["Andere Staatsangehörigkeit (%)"].append(andere_row.iloc[:, 2:].mean(axis=1).values[0])


df_staatsangehoerigkeit = pd.DataFrame(staatsangehoerigkeit).sort_values(by="Jahr")
df_staatsangehoerigkeit["Jahr"] = df_staatsangehoerigkeit["Jahr"].astype(int)

In [ ]:
fig = px.area(
    df_staatsangehoerigkeit, 
    x="Jahr", 
    y=["Andere Staatsangehörigkeit (%)", "Deutsche Staatsangehörigkeit (%)"], 
    title="Wachsende Diversität: Immer mehr Studierende mit <span style='color:#006c31;'>anderer Staatsangehörigkeit</span>",
    subtitle="Entwicklung der Studierendenbefragung zwischen 2013 und 2024 im FB09",
    labels={"value": "Anteil (%)", "variable": "Staatsangehörigkeit"},
    color_discrete_sequence=[px.colors.qualitative.Safe[3], px.colors.qualitative.Safe[4]],
    template="infoviz"
)

last_year = df_staatsangehoerigkeit["Jahr"].max()
last_andere = df_staatsangehoerigkeit[df_staatsangehoerigkeit["Jahr"] == last_year]["Andere Staatsangehörigkeit (%)"].values[0]


fig.add_annotation(
    x=last_year-1.08, 
    y=last_andere+8,  
    text=f"Andere Staatsangehörigkeit<br>{last_andere:.1f}% (2024)",
    showarrow=False,
    align="right",
    font=dict(size=12, color=px.colors.qualitative.Safe[3], weight="bold")
)

fig.update_layout(
    yaxis=dict(range=[0, 100], title="Anteil Studierende (%)"),  # Y-Achsenbereich festlegen
    xaxis=dict(range=[2013, 2024.05], title="", dtick=1),
    showlegend=False
)

fig.show()

## Bachelor vs Master

In [ ]:
file_path = "data.xlsx"
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")
sheets.pop(next(iter(sheets)))  

staatsangehoerigkeit = {"Jahr": [], "Bachelor (%)": [], "Master (%)": []}

for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()
    
    bachelor_studiengaenge = [col for col in df.columns if "Bachelor" in col]
    master_studiengaenge = [col for col in df.columns if "Master" in col]

    andere_row = df[df["Variable"].str.contains("Staatsangehörigkeit", na=False) & 
                    df["Category"].str.contains("Andere Staatsangehörigkeit", na=False)]
    
    if not andere_row.empty:
        bachelor_mean = andere_row[bachelor_studiengaenge].mean(axis=1).values[0]
        master_mean = andere_row[master_studiengaenge].mean(axis=1).values[0]

        staatsangehoerigkeit["Jahr"].append(jahr)
        staatsangehoerigkeit["Bachelor (%)"].append(bachelor_mean)
        staatsangehoerigkeit["Master (%)"].append(master_mean)

df_staatsangehoerigkeit = pd.DataFrame(staatsangehoerigkeit).sort_values(by="Jahr")
df_staatsangehoerigkeit["Jahr"] = df_staatsangehoerigkeit["Jahr"].astype(int)

df_melted = df_staatsangehoerigkeit.melt(id_vars="Jahr", var_name="Studienabschluss", value_name="Anteil Andere Staatsangehörigkeit (%)")

In [ ]:
import plotly.express as px

fig = px.line(
    df_melted,
    x="Jahr",
    y="Anteil Andere Staatsangehörigkeit (%)",
    color="Studienabschluss",
    markers=True, 
    title="Internationalisierungstrend besonders in den <span style='color:#cc5b6e;'>Masterstudiengängen</span> sichtbar",
    subtitle="Entwicklung der Studierendenbefragung zwischen 2013 und 2024 im FB09",
    template="infoviz",
    color_discrete_sequence=px.colors.qualitative.Safe
)

fig.update_traces(line=dict(width=3))

last_year = df_melted["Jahr"].max()

last_bachelor_value = df_melted[(df_melted["Jahr"] == last_year) & (df_melted["Studienabschluss"] == "Bachelor (%)")].iloc[0]["Anteil Andere Staatsangehörigkeit (%)"]
last_master_value = df_melted[(df_melted["Jahr"] == last_year) & (df_melted["Studienabschluss"] == "Master (%)")].iloc[0]["Anteil Andere Staatsangehörigkeit (%)"]

fig.add_annotation(
    x=last_year, 
    y=last_bachelor_value + 2,
    text=f"Bachelor {last_bachelor_value:.2f}%", 
    showarrow=False, 
    font=dict(size=14, color=px.colors.qualitative.Safe[0], weight="bold")
)

fig.add_annotation(
    x=last_year, 
    y=last_master_value + 2, 
    text=f"Master {last_master_value:.2f}%", 
    showarrow=False, 
    font=dict(size=14, color=px.colors.qualitative.Safe[1], weight="bold")
)

fig.update_layout(
    yaxis=dict(range=[-0.08, 55], title="Studierende anderer Staatsangehörigkeit (%)"),
    xaxis=dict(range=[2012.9, 2024.05], title="", dtick=1),
    showlegend=False 
)

fig.show()